In [44]:
import pandas as pd

df = pd.DataFrame({
    'ID': [1,2,3,1,2,3],
    'Values': list(range(6)),
})
df.index = range(1, len(df) + 1)
df.drop('ID', inplace=True, axis=1)
df.index.name = 'ID'
df

,Values
ID,
1,0
2,1
3,2
4,3
5,4
6,5


In [37]:
df.reset_index(inplace=True, drop=True)
df.index = df.index + 1
df.index.rename('ID', inplace = True)
df

,Values
ID,
1,0
2,1
3,2
4,3
5,4
6,5


In [ ]:
df2 = pd.DataFrame({
    'ID': [1,2,3,1,2,3],
    'Values': list(range(6)),
})
df.set_index('ID', inplace=True)
df

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime

DATASET_FILEPATH = 'Taxi_Chicago.csv'
df = pd.read_csv(DATASET_FILEPATH)
df = df[~df['Taxi ID'].isnull()]

server = 'DESKTOP-HFN3ITE'
database = 'TaxiTrips'
username = 'user1'
password = 'sa'
driver = 'ODBC Driver 17 for SQL Server'
connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver}'
engine = create_engine(connection_string)

df['ID_TaxiT'] = pd.factorize(df['Taxi ID'])[0] + 1
taxi_table = df[['ID_TaxiT', 'Taxi ID']].drop_duplicates().rename(columns={'Taxi ID': 'HashIDTaxi'})
taxi_table.set_index('ID_TaxiT', inplace=True)
taxi_table.to_sql('Taxi', con=engine, schema='stageProjectIDH', if_exists='append', index=True)

df['ID_PaymentTypeT'] = pd.factorize(df['Payment Type'])[0] + 1
payment_type_table = df[['ID_PaymentTypeT', 'Payment Type']].drop_duplicates().rename(columns={'Payment Type': 'Title'})
payment_type_table.set_index('ID_PaymentTypeT', inplace=True)
payment_type_table.to_sql('PaymentType', con=engine, schema='stageProjectIDH', if_exists='append', index=True)

df['ID_CompanyT'] = pd.factorize(df['Company'])[0] + 1
company_table = df[['ID_CompanyT', 'Company']].drop_duplicates().rename(columns={'Company': 'Title'})
company_table.set_index('ID_CompanyT', inplace=True)
company_table.to_sql('Company', con=engine, schema='stageProjectIDH', if_exists='append', index=True)

def normalize_timestamps(df, column_name, offset):
    type_ = column_name.split()[1]
    df[f'ID_TimestampT_{type_}'] = pd.factorize(df[column_name].astype(str))[0] + 1
    timestamp_fields = df[column_name].apply(lambda x: datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p'))
    timestamp_table = pd.DataFrame({
        'ID_TimestampT': pd.factorize(df[column_name].astype(str))[0] + 1,
        'Year': timestamp_fields.dt.year,
        'Month': timestamp_fields.dt.month,
        'Day': timestamp_fields.dt.day,
        'Hour': timestamp_fields.dt.hour,
        'Minute': timestamp_fields.dt.minute,
        'Second': timestamp_fields.dt.second
    }).drop_duplicates()
    timestamp_table.set_index('ID_TimestampT', inplace=True)
    return timestamp_table

pickup_timestamps = normalize_timestamps(df, 'Trip Start Timestamp')
pickup_timestamps.to_sql('TimestampT', con=engine, schema='stageProjectIDH', if_exists='append', index=False)

dropoff_timestamps = normalize_timestamps(df, 'Trip End Timestamp')
dropoff_timestamps.to_sql('TimestampT', con=engine, schema='stageProjectIDH', if_exists='append', index=False)

all_timestamps = pd.concat([pickup_timestamps, dropoff_timestamps]).drop_duplicates().reset_index(drop=True)
all_timestamps.to_sql('TimestampT', con=engine, schema='stageProjectIDH', if_exists='append', index=False)

def normalize_areas(df, column_name):
    area_table = pd.DataFrame({
        'ID_CommunityAreaT': pd.factorize(df[column_name])[0] + 1,
        'IdentifierArea': df[column_name]
    })
    area_table.set_index('ID_CommunityAreaT', inplace=True)
    return area_table

pickup_areas = normalize_areas(df, 'Pickup Community Area')
pickup_areas.to_sql('CommunityArea', con=engine, schema='stageProjectIDH', if_exists='append', index=False)

dropoff_areas = normalize_areas(df, 'Dropoff Community Area')
dropoff_areas.to_sql('CommunityArea', con=engine, schema='stageProjectIDH', if_exists='append', index=False)

all_areas = pd.concat([pickup_areas, dropoff_areas]).drop_duplicates().reset_index(drop=True)
all_areas.to_sql('CommunityArea', con=engine, schema='stageProjectIDH', if_exists='append', index=False)

def normalize_tracts(df, column_name):
    tract_table = pd.DataFrame({
        'ID_CensusTractT': pd.factorize(df[column_name])[0] + 1,
        'IdentifierCensusTract': df[column_name]
    })
    tract_table.set_index('ID_CommunityAreaT', inplace=True)
    return tract_table
    tracts = df[column_name].dropna().drop_duplicates().astype(int).reset_index(drop=True)
    tract_table = pd.DataFrame(tracts)
    tract_table['ID_CensusTractT'] = tract_table.index + 1
    tract_table.columns = ['IdentifierCensusTract', 'ID_CensusTractT']
    return tract_table

pickup_tracts = normalize_tracts(df, 'Pickup Census Tract')
pickup_tracts.to_sql('CensusTract', con=engine, schema='stageProjectIDH', if_exists='append', index=False)

dropoff_tracts = normalize_tracts(df, 'Dropoff Census Tract')
dropoff_tracts.to_sql('CensusTract', con=engine, schema='stageProjectIDH', if_exists='append', index=False)

# Combine and deduplicate census tracts
all_tracts = pd.concat([pickup_tracts, dropoff_tracts]).drop_duplicates().reset_index(drop=True)
all_tracts.to_sql('CensusTract', con=engine, schema='stageProjectIDH', if_exists='append', index=False)

# Insert data for Trip
# Map the normalized IDs to the Trip table
df = df.merge(all_timestamps, left_on='Trip Start Timestamp', right_on='Timestamp', how='left')
df = df.merge(all_timestamps, left_on='Trip End Timestamp', right_on='Timestamp', how='left', suffixes=('_Start', '_End'))
df = df.merge(all_areas, left_on='Pickup Community Area', right_on='IdentifierArea', how='left')
df = df.merge(all_areas, left_on='Dropoff Community Area', right_on='IdentifierArea', how='left', suffixes=('_Pickup', '_Dropoff'))
df = df.merge(all_tracts, left_on='Pickup Census Tract', right_on='IdentifierCensusTract', how='left')
df = df.merge(all_tracts, left_on='Dropoff Census Tract', right_on='IdentifierCensusTract', how='left', suffixes=('_Pickup', '_Dropoff'))

# Prepare the Trip table data
trip_table = df.rename(columns={
    'Trip ID': 'IdentifierTrip',
    'ID_TaxiT': 'ID_Taxi',

    'ID_PaymentTypeT': 'ID_PaymentType',
    'ID_CompanyT': 'ID_Company',

    'Trip Seconds': 'Seconds',
    'Trip Miles': 'Miles',
    'Fare': 'Fares',
    'Trip Total': 'Total',

    'ID_TimestampT_Start': 'StartTimestampID',
    'ID_TimestampT_End': 'EndTimestampID',
    'ID_CommunityAreaT_Pickup': 'PickupCommunityAreaID',
    'ID_CommunityAreaT_Dropoff': 'DropoffCommunityAreaID',
    'ID_CensusTractT_Pickup': 'PickupCensusTractID',
    'ID_CensusTractT_Dropoff': 'DropoffCensusTractID'
}).drop(columns=['Taxi ID', 'Payment Type', 'Company', 'Trip Start Timestamp', 'Trip End Timestamp', 'Pickup Community Area', 'Dropoff Community Area', 'Pickup Census Tract', 'Dropoff Census Tract', 'Timestamp', 'IdentifierArea', 'IdentifierCensusTract'])  # Drop redundant columns after ID mapping
trip_table['ID_Trip'] = pd.factorize(trip_table['IdentifierTrip'])[0] + 1
trip_table.to_sql('Trip', con=engine, schema='stageProjectIDH', if_exists='append', index=True)


KeyboardInterrupt: 